Information comes from two sources:
1. the CSV file containing the schedule for the grid search
2. the DeepRacer evaluation JSON files 

In [31]:
import pandas as pd
import glob
import json

In [32]:
vers = 'v1'
datestr = '030821'

Load the CSV file containing the schedule for the grid search.

In [33]:
deep_racer_grid = pd.read_csv('deep_racer_grid.csv')
deep_racer_grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Index                100 non-null    int64  
 1   batch_size           100 non-null    int64  
 2   epochs               100 non-null    int64  
 3   learning_rate        100 non-null    float64
 4   entropy              100 non-null    float64
 5   discount_factor      100 non-null    float64
 6   loss_type            100 non-null    object 
 7   experience_episodes  100 non-null    int64  
 8   car                  100 non-null    object 
 9   id                   100 non-null    object 
 10  assign               100 non-null    object 
dtypes: float64(3), int64(4), object(4)
memory usage: 8.7+ KB


In [34]:
# Retain only pertinent columns in the data frame.
# I am dropping 'car' because the same car is always used (for now).
deep_racer_grid = deep_racer_grid[['id', 'Index', 'batch_size', 'epochs', 'learning_rate', 'entropy', 'discount_factor',
                                   'loss_type', 'experience_episodes', 'assign']]
deep_racer_grid.head()

,id,Index,batch_size,epochs,learning_rate,entropy,discount_factor,loss_type,experience_episodes,assign
0,bhi1,1,64,3,0.0003,0.0010,0.9999,mse,20,Joe
1,bhi2,2,32,10,0.0010,0.0010,0.9900,huber,10,Joe
2,bhi3,3,32,8,0.0003,0.0001,0.9900,mse,20,Joe
3,bhi4,4,32,10,0.0010,0.0100,0.9900,mse,10,Joe
4,bhi5,5,64,10,0.0003,0.0100,0.9999,huber,20,Joe


In [35]:
# Create list of BHI ids.
bhi_id_list = deep_racer_grid['id'].unique().tolist()

In [36]:
# Initialize temporary output data frame.
tmp_out_df = None

# Loop over BHI ids.
for bhi_id in bhi_id_list:

    # Make a list of JSON files that match the current id.
    glob_list = glob.glob(f'evaluations\{bhi_id}\metrics\evaluation\*.json')

    # Reference: https://www.geeksforgeeks.org/read-json-file-using-python/
    if len(glob_list) > 0:

        # Loop over JSON files that we found.
        for j in glob_list:
            # Open JSON file 
            f = open(j)

            # returns JSON object as a dictionary 
            data = json.load(f) 

            # Iterating through the json list.
            for i, d in enumerate(data['metrics']):
                tmp_df = pd.Series(d).to_frame().transpose()
                if i == 0:
                    id_df = tmp_df
                else:
                    id_df = id_df.append(tmp_df)

            # Closing file 
            f.close()

        # Convert episode_status to numeric representation.
        id_df['lap_complete'] = id_df['episode_status'].apply(lambda x: 1 if x=='Lap complete' else 0)
        id_df.drop('episode_status',axis=1)

        # Average metrics across trials within current id.
        # Convert Series to DataFrame with Series indices as column names.
        # Reference: https://stackoverflow.com/a/40224468
        mean_out_series = id_df.mean(axis=0)
        mean_out_df = pd.DataFrame(mean_out_series).transpose()

        # Add 'id' as the index.
        col_names = mean_out_df.columns.tolist()
        mean_out_df['id'] = bhi_id
        #mean_out_df.set_index('id',inplace=True)
        
        # Put the 'id' column first.
        mean_out_df = mean_out_df[['id',
                                   'off_track_count',
                                   'reversed_count',
                                   'trial',
                                   'reset_count',
                                   'completion_percentage',
                                   'elapsed_time_in_milliseconds',
                                   'immobilized_count',
                                   'metric_time',
                                   'crash_count',
                                   'start_time',
                                   'lap_complete']]

        # Add average metrics to the temporary output data frame.
        if tmp_out_df is None:
            tmp_out_df = mean_out_df
        else:
            tmp_out_df = tmp_out_df.append(mean_out_df, ignore_index=True)

In [37]:
# Now merge the temporary output data frame with the main output data frame.
deep_racer_grid = pd.merge(deep_racer_grid, tmp_out_df, on='id', how='outer')
deep_racer_grid

,id,Index,batch_size,epochs,learning_rate,entropy,discount_factor,loss_type,experience_episodes,assign,...,reversed_count,trial,reset_count,completion_percentage,elapsed_time_in_milliseconds,immobilized_count,metric_time,crash_count,start_time,lap_complete
0,bhi1,1,64,3,0.000300,0.0010,0.9999,mse,20,Joe,...,0.0,3.0,0.8,68.8,9491.4,0.0,27875.0,0.0,18383.6,0.2
1,bhi2,2,32,10,0.001000,0.0010,0.9900,huber,10,Joe,...,0.0,3.0,0.6,69.2,9589.8,0.0,34835.0,0.0,25245.2,0.4
2,bhi3,3,32,8,0.000300,0.0001,0.9900,mse,20,Joe,...,0.0,3.0,0.4,95.4,11883.4,0.0,43260.2,0.0,31376.8,0.6
3,bhi4,4,32,10,0.001000,0.0100,0.9900,mse,10,Joe,...,0.0,3.0,0.6,59.4,7617.4,0.0,30102.0,0.0,22484.6,0.4
4,bhi5,5,64,10,0.000300,0.0100,0.9999,huber,20,Joe,...,0.0,3.0,1.0,65.8,8996.6,0.0,37954.0,0.0,28957.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bhi96,96,64,8,0.000003,0.0010,0.9999,mse,10,Dan L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,bhi97,97,128,3,0.000030,0.0100,0.9999,huber,50,Dan L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,bhi98,98,64,10,0.000003,0.0001,0.9990,huber,10,Dan L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,bhi99,99,64,3,0.000030,0.0001,0.9990,huber,10,Dan L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Write to CSV file.
outFile = 'DeepRacer Grid - ' + vers + '.' + datestr + '.csv'
deep_racer_grid.to_csv(outFile,index=False)

PermissionError: [Errno 13] Permission denied: 'DeepRacer Grid - v1.030821.csv'

In [ ]:
deep_racer_grid[~deep_racer_grid.trial.isna()].shape

In [ ]:
deep_racer_grid[completion_percentage]